In [18]:
import os
import tarfile
import requests
%load_ext autoreload
%autoreload 2
files = [
    {
        "url": "https://msmarco.z22.web.core.windows.net/msmarcoranking/collection.tar.gz",
        "name": "collection.tar.gz"
    },
    {
        "url": "https://msmarco.z22.web.core.windows.net/msmarcoranking/queries.tar.gz",
        "name": "queries.tar.gz"
    },
    {
        "url": "https://msmarco.z22.web.core.windows.net/msmarcoranking/qrels.dev.tsv",
        "name": "qrels.dev.tsv"
    }
]

for file in files:
    if not os.path.exists(file["name"].replace('.tar.gz', '.tsv')):
        response = requests.get(file["url"])
        with open(file["name"], 'wb') as f:
            f.write(response.content)
        if file["name"].endswith('.tar.gz'):
            with tarfile.open(file["name"], 'r:gz') as tar:
                tar.extractall(path='.')

/tmp/ipykernel_2700/2838203614.py:28: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(path='.')


In [5]:
from load_corpus import read_collection, read_queries_dev
import pandas as pd

df = read_collection(limit=500000)
dev = read_queries_dev()
qrels = pd.read_csv("qrels.dev.tsv", sep="\t", names=["qid","_","pid","rel"], dtype={"qid":str,"pid":str,"rel":int})
dev_with_pids = dev.merge(qrels[["qid","pid"]], on="qid", how="inner").drop_duplicates(["qid","pid"])

dev_filtered: pd.DataFrame = dev_with_pids[dev_with_pids["pid"].astype(str).isin(df["pid"].astype(str))].copy()
dev_filtered.head()

Reading collection: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500000/500000 [00:00<00:00, 587624.69it/s]


In [19]:
from index_bm25 import build_bm25

build_bm25(df)

Tokenizing: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500000/500000 [00:40<00:00, 12372.02it/s]


In [6]:
from index_bm25 import write_trec_run

write_trec_run(dev_filtered, "runs/bm25.dev.trec", topk=1000)

Processing queries: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1014/1014 [07:24<00:00,  2.28it/s]


In [3]:
from ranx import Qrels, Run, evaluate
qrels = Qrels.from_file("qrels.dev.tsv", kind="trec")
run   = Run.from_file("runs/bm25.dev.trec", kind="trec")
print(evaluate(qrels, run, metrics=["ndcg@10","map@10","recall@100"], make_comparable=True))

/home/zuber_purahoo2/datascience-venv/lib/python3.13/site-packages/ranx/metrics/ndcg.py:72: NumbaTypeSafetyWarning: unsafe cast from uint64 to int64. Precision may be lost.
  scores[i] = _ndcg(qrels[i], run[i], k, rel_lvl, jarvelin)


{'ndcg@10': np.float64(0.003570830233922889), 'map@10': np.float64(0.0027275552105520797), 'recall@100': np.float64(0.012281538258543548)}
